In [5]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers
import pickle
from sklearn.model_selection import train_test_split
import keras.backend as K
import random
from keras.models import load_model
from MadsNeural_network_500epoch import *

WIRE_ANTENNA = False


In [10]:

if WIRE_ANTENNA == True:
    file_path = "Data/WIRE_results/WIRE_Forward_results_with_band_centre.pkl"
else:
    file_path = "Data/MIFA_results/MIFA_Forward_results_with_band_centre.pkl"

# Load data
with open(file_path, 'rb') as f:
    data_to_load = pickle.load(f)

print(data_to_load.keys())

bandwidth = data_to_load["bandwidth"]
center_frequency = data_to_load["centre_frequency"]
f1f2 = data_to_load["f1f2"]

parameter = data_to_load["Parameter combination"]

# #Normalize the data wrt. distribution
# parameter_norm = normalize_data(parameter,np.mean(parameter),np.std(parameter), False)
# bandwidth_norm = normalize_data(bandwidth,np.mean(bandwidth),np.std(bandwidth), False)
# center_frequency_norm = normalize_data(center_frequency,np.mean(center_frequency),np.std(center_frequency), False)

input_vector =  np.asarray([[bandwidth[x], center_frequency[x]] for x in range(len(bandwidth))])
input_vector2 = np.asarray([np.concatenate((input_vector[x], f1f2[x])) for x in range(len(input_vector))])
output_vector = np.asarray(parameter)

print(f'Input vector shape: {input_vector.shape}')
print(f'Output vector shape:{output_vector.shape}')

# Split data into training and testing
x_train, x_test, y_train, y_test = train_test_split(input_vector, output_vector, test_size=0.3, random_state=42)

#Normalize the data
x_train_norm = normalize_data(x_train,np.mean(x_train),np.std(x_train), False)
x_test_norm = normalize_data(x_train,np.mean(x_train),np.std(x_train), False)
y_train_norm = normalize_data(y_train,np.mean(y_train),np.std(y_train), False)
y_test_norm = normalize_data(y_test,np.mean(y_test),np.std(y_test), False)

print(len(parameter))
print(len(input_vector))
print(x_train_norm.shape)
print(y_train_norm.shape)

dict_keys(['Parameter combination', 'S1,1', 'Frequency', 'bandwidth', 'centre_frequency', 'f1f2'])
Input vector shape: (180000, 2)
Output vector shape:(180000, 4)
180000
180000
(54000, 2)
(126000, 4)


In [7]:
#Create the model
model = keras.Sequential([
    layers.InputLayer(input_shape=(x_train.shape[1])),
    layers.Dense(256, activation='relu', name = 'layer1'),
    layers.Dense(256, activation='relu', name = 'layer2'),
    layers.Dense(256, activation='relu', name = 'layer3'),
    layers.Dense(256, activation='relu', name = 'layer4'),
    layers.Dense(256, activation='relu', name = 'layer5'),
    layers.Dense(y_train.shape[1], activation = 'linear', name = 'Output_layer')
])
model.summary()
# Compile model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=[keras.metrics.MeanSquaredError()]
              )

#Train model
history = model.fit(x_train_norm,
                    y_train_norm,
                    epochs=500,
                    batch_size=200,)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 256)               768       
                                                                 
 layer2 (Dense)              (None, 256)               65792     
                                                                 
 layer3 (Dense)              (None, 256)               65792     
                                                                 
 layer4 (Dense)              (None, 256)               65792     
                                                                 
 layer5 (Dense)              (None, 256)               65792     
                                                                 
 Output_layer (Dense)        (None, 4)                 1028      
                                                                 
Total params: 264964 (1.01 MB)
Trainable params: 264964 

ValueError: Data cardinality is ambiguous:
  x sizes: 36000
  y sizes: 144000
Make sure all arrays contain the same number of samples.

In [ ]:
plt.rcParams.update({'font.family': 'serif', 'font.serif': ['Times New Roman'], 'font.size': 16})      
# Define the loss and accuracy for the training and test data
loss_train.extend(model.history.history['loss'])
mean_error_train.extend(model.history.history['mean_squared_error'])
if PLOT_TRAIN:
    if WIRE_ANTENNA:
        plt.figure(figsize=(10,5))
        plt.subplot(121)
        plt.plot(np.array(loss_train).T)
        plt.ylabel('Absolute error')
        plt.xlabel('epoch')
        plt.legend(['Absolute error'])
        plt.ylim([0, 0.7])
        plt.subplot(122)
        plt.plot((np.array(mean_error_train).T))
        plt.ylabel('Mean-squared error')
        plt.xlabel('epoch')
        plt.legend(['Mean-squared error'])
        #plt.ylim([0, 0.7])
        plt.show()
        # plt.savefig(f'data/DNN_results_reversal/train_loss/loss_{(j+1)*100}.png')
        plt.close()


In [ ]:
# Run the model on the test data and get the loss and mean-squared error
y_pred_norm = model.predict(x_test_norm)
_ , mean_error_pred = model.evaluate(x_test_norm, y_test_norm)

#Reverse normalization
# Reverse the normalization of the data individually
print(f"y_pred_norm shape: {y_pred_norm.shape}")
y_pred = normalize_data(y_pred_norm, np.mean(y_train), np.std(y_train), True)


PLOT_TEST = True
# Plot the testing results
if PLOT_TEST:
    if WIRE_ANTENNA:
        # Make a grouped bar plot with the predicted parameters and the test parameters
        random_indices = random.sample(range(0, y_pred.shape[0]), 4)
        plt.figure(figsize=(20, 5))
        width = 0.1  # the width of the bars
        labels = ["Wire length", "Wire height", "Wire thickness"]
        for idx, i in enumerate(random_indices):
            plt.subplot(2, 2, idx+1)
            plt.grid(True)
            bars1 = plt.bar(np.arange(1,4) - width/2, y_test[i]/y_test[i], width)
            bars2 = plt.bar(np.arange(1,4) + width/2, y_pred[i]/y_test[i], width)
            plt.xticks(np.arange(1,4), labels)
            plt.legend([bars1, bars2], ['Test', 'Pred'])
        
    else:
        random_indices = random.sample(range(0, y_pred.shape[0]), 4)
        plt.figure(figsize=(20, 5))
        width = 0.1  # the width of the bars
        labels = ['Tw1','groundingPinTopLength','Line1_height','substrateH']
        for idx, i in enumerate(random_indices):
            plt.subplot(2, 2, idx+1)
            plt.grid(True)
            bars1 = plt.bar(np.arange(1,5) - width/2, y_test[i]/y_test[i], width)
            bars2 = plt.bar(np.arange(1,5) + width/2, y_pred[i]/y_test[i], width)
            plt.xticks(np.arange(1,5), labels)
            plt.legend([bars1, bars2], ['Test', 'Pest'])
    plt.show()


In [ ]:
# Save model
if WIRE_ANTENNA:
    model.save('data/Wire_Results/Reverse2ForwardWire_model.keras')
else:
    model.save('data/MIFA_results/Reverse2ForwardMIFA_model.keras')